# Optimal Power Flow for Active Distribution Grids

In [ ]:
import pfnet
import optalg
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Parse network

In [ ]:
num_periods = 24
parser = pfnet.ParserMAT()
net = parser.parse('../../cases/case39.mat',num_periods)

## Generator ramps

In [ ]:
for gen in net.generators:
        gen.dP_max = 0.3*(gen.P_max-gen.P_min)

## Demand response

In [ ]:
for load in net.loads:
        load.P_max = 1.1*load.P[0]
        load.P_min = 0.9*load.P[0]
        load.target_power_factor = 0.9

## Renewable energy sources

In [ ]:
wind_profile = 0.5*(1+np.cos(np.linspace(0.,2.*np.pi,num_periods)))
net.add_var_generators(net.get_load_buses(),50.,100.,0.,0.,0.)
for gen in net.var_generators:
        gen.P = gen.P[0]*wind_profile
        gen.Q_max = 0.1*np.max(gen.P)
        gen.Q_min = -0.1*np.max(gen.P)

## Batteries

In [ ]:
net.add_batteries(net.get_generator_buses(),10.,40.,1.,1.)

## Construct problem

In [ ]:
net.set_flags('bus',['variable','bounded'],'any','voltage magnitude')
net.set_flags('bus','variable','not slack','voltage angle')
net.set_flags('generator',['variable','bounded'],'any',['active power','reactive power'])
net.set_flags('variable generator',['variable','bounded'],'any',['active power','reactive power'])
net.set_flags('battery',['variable','bounded'],'any',['charging power','energy level'])
net.set_flags('load',['variable','bounded'],'any',['active power','reactive power'])

problem = pfnet.Problem()
problem.set_network(net)
problem.add_function(pfnet.Function('generation cost',1.,net))
problem.add_constraint(pfnet.Constraint('AC power balance',net))
problem.add_constraint(pfnet.Constraint('battery dynamics',net))
problem.add_constraint(pfnet.Constraint('variable bounds',net))
problem.add_constraint(pfnet.Constraint('load constant power factor',net))
problem.add_constraint(pfnet.Constraint('generator ramp limits',net))
problem.analyze()
problem.show()

## Solve problem

In [ ]:
solver = optalg.opt_solver.OptSolverAugL()
solver.set_parameters({'quiet':True})
solver.solve(problem)
print(solver.get_status(), solver.get_iterations())

## Visualize solution